![dss-jakuczun](images/Jakuczun.png)

# Why is optimization important?

## What is optimization model?

![opt-model](images/opt-model.png)

## **recommendation = optimize(forecast)**

![opt-value-chain](images/opt-model-value-chain.png)

# CRM campaign use case

## What is a business problem we want to solve?

### Key questions

#### What should we offer?

Company offers **many products/services** to their customers.

#### How should we offer the product?

Company uses **different communication channels** to reach their customers. For example: email, phone, www.

Company can also decide on time and message when reaching particular customer.

#### Whom should we offert the product?

Depending on customer profile (e.g. demography, behavioral profile, etc.)the reaction to the offering for a given product via a given channel can differ.

### Business process

![kampania_process](images/cmpgn_process.png)

##### Estimating probability of buying

Proper place for *machine learning* model. The model for each tupe `(customer, product, channel)` estimates probabilty of buying (or any positive action).

##### Selecting optimal strategy of conducting the campaign

Some constraints are given:

* Cap on campaing budget.
* Minimal product exposure (number of offers).
* Minimum ROI must be achieved.

**Here *machine learning* cannot help!**

# What is `minizinc`?

## [MiniZinc](https://www.minizinc.org/index.html) is a high-level **constraint modelling language**.

![model-example](images/minizinc-model-example.png)

## It separates model declaration from data and solver.

* File with model with `mzn` extension
* File with data with `dzn` extension or `json`

In [73]:
!minizinc --solver Gecode dss-marketing-optimization.mzn small_dataset.json

vAssignment = array3d(1..2, 1..2, 1..2, [1, 0, 1, 0, 1, 0, 0, 1]);
----------


## Many solvers are available

![minzinc-solvers](images/minzinc-solvers.png)

## It has a rich library of constraints for many practical problems.

![minizinc-constraints](images/minizinc-global-constraints.png)

## It is an extensible and highly powerful language

![minizinc-modular](images/minizinc-modular.png)

![minizinc-functions](images/minizinc-functions.png)

![minizinc-predicates](images/minizinc-predicates.png)

![minizinc-let](images/minizinc-let.png)

# Optimization model for CRM Campaign

## Packages instalation

In [1]:
!cat requirements.txt

iminizinc
ipykernel
minizinc
nest_asyncio
argparse


In [2]:
!pip3 install -r requirements.txt

You should consider upgrading via the '/workspaces/conf-dss2021/venv/bin/python -m pip install --upgrade pip' command.


## `minizinc` extension for `jupyter notebook`

In [3]:
%load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.5.3
Copyright (C) 2014-2020 Monash University, NICTA, Data61


## Model parameters

### Customers, producs and channels

There are:

* `pCustomerN` customer
* `pProductsN` products/services
* `pChannelsN` channels

In `minizinc` we declare it as follows

In [4]:
%%minizinc
int: pCustomersN;
int: pProductsN;
int: pChannelsN;

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined


### Expected reve for product in channel for a customer

Expected reve when given product is offered to given customer via given channel. We store this information in 3 dimensional array.

In [5]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 
      1..pProductsN, 
      1..pChannelsN] of float: pExpectedReve;

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined


### Minimal guaranteed ROI

In [6]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
float: pMinHurdleReve;

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pMinHurdleReve is undefined


### Marketing costs

Marketing costs depend only on a channel.

In [7]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
float: pMinHurdleReve;
array[1..pChannelsN] of float: pChannelCosts;

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pMinHurdleReve is undefined
Variable pChannelCosts is undefined


### Maximum budget


In [8]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
float: pMinHurdleReve;
array[1..pChannelsN] of float: pChannelCosts;

float: pMaxBudget;

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pMinHurdleReve is undefined
Variable pChannelCosts is undefined
Variable pMaxBudget is undefined


### Minimum exposure of a product 

In [9]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
float: pMinHurdleReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget;

array[1..pProductsN] of int: pProductsMinOffersN;

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pMinHurdleReve is undefined
Variable pChannelCosts is undefined
Variable pMaxBudget is undefined
Variable pProductsMinOffersN is undefined


## Variables

### Assigning product to customer and channel

For each customer, product and channel decided if we should act. We will denote this variable as `vAssignment[customer, product, channel]`.

In [10]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN,
      1..pProductsN,
      1..pChannelsN] of var 0..1: vAssignment;


Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pChannelCosts is undefined
Variable pMaxBudget is undefined
Variable pProductsMinOffersN is undefined
Variable pMinHurdleReve is undefined


## Constraints

### Maximum budget

Total costs of contacting customers cannot be more than `pMaxBudget`.

In [11]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of var 0..1: vAssignment;

%Constraints
constraint sum(customer in 1..pCustomersN,
               product in 1..pProductsN,
               channel in 1..pChannelsN)(
                   vAssignment[customer, product, channel]*
                   pChannelCosts[channel]
               ) <= pMaxBudget;

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pChannelCosts is undefined
Variable pMaxBudget is undefined
Variable pProductsMinOffersN is undefined
Variable pMinHurdleReve is undefined


### Minimum exposure for each product

Each product must be offered given (`pProductsMinOfferN`) times.

In [12]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of var 0..1: vAssignment;

%Constraints
constraint sum(customer in 1..pCustomersN, product in 1..pProductsN, channel in 1..pChannelsN)(vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]) <= pMaxBudget;

constraint forall(product in 1..pProductsN) (
    sum(customer in 1..pCustomersN,
        channel in 1..pChannelsN) (
              vAssignment[customer, product, channel]
    ) >= pProductsMinOffersN[product]);

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pChannelCosts is undefined
Variable pMaxBudget is undefined
Variable pProductsMinOffersN is undefined
Variable pMinHurdleReve is undefined


### Minimum guaranteed ROI 

Company wants to have minimum ROI.

In [13]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of var 0..1: vAssignment;

%Constraints
constraint sum(customer in 1..pCustomersN, product in 1..pProductsN, channel in 1..pChannelsN)(vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]) <= pMaxBudget;

constraint forall(product in 1..pProductsN) (
    sum(customer in 1..pCustomersN,
        channel in 1..pChannelsN) (
              vAssignment[customer, product, channel]
    ) >= pProductsMinOffersN[product]);

constraint sum(customer in 1..pCustomersN,
               product in 1..pProductsN,
               channel in 1..pChannelsN)
            (
                vAssignment[customer, product, channel]*pExpectedReve[customer, product, channel]
            ) >= 
            (1+pMinHurdleReve)*sum(customer in 1..pCustomersN,
                                   product in 1..pProductsN,
                                   channel in 1..pChannelsN)
                                    (
                                        vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]
                                    );

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pChannelCosts is undefined
Variable pMaxBudget is undefined
Variable pProductsMinOffersN is undefined
Variable pMinHurdleReve is undefined


## Objective function

### We want to optimize total margin from the campaign.

In [14]:
%%minizinc
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of var 0..1: vAssignment;

%Constraints
constraint sum(customer in 1..pCustomersN, product in 1..pProductsN, channel in 1..pChannelsN)(vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]) <= pMaxBudget;

constraint forall(product in 1..pProductsN) (
    sum(customer in 1..pCustomersN,
        channel in 1..pChannelsN) (
              vAssignment[customer, product, channel]
    ) >= pProductsMinOffersN[product]);

constraint sum(customer in 1..pCustomersN,
               product in 1..pProductsN,
               channel in 1..pChannelsN)
            (
                vAssignment[customer, product, channel]*pExpectedReve[customer, product, channel]
            ) >= 
            (1+pMinHurdleReve)*sum(customer in 1..pCustomersN,
                                   product in 1..pProductsN,
                                   channel in 1..pChannelsN)
                                    (
                                        vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]
                                    );

solve maximize sum(customer in 1..pCustomersN,
                   product in 1..pProductsN,
                   channel in 1..pChannelsN)
                (
                    vAssignment[customer, product, channel]*
                    pExpectedReve[customer, product, channel]
                );

Variable pCustomersN is undefined
Variable pProductsN is undefined
Variable pChannelsN is undefined
Variable pExpectedReve is undefined
Variable pChannelCosts is undefined
Variable pMaxBudget is undefined
Variable pProductsMinOffersN is undefined
Variable pMinHurdleReve is undefined


## Finding Solution

### Using external datasets

To provide values for model parameters we can use `json` format.

In [15]:
!jq '.' small_dataset.json

{
  "pCustomersN": 2,
  "pProductsN": 2,
  "pChannelsN": 2,
  "pMaxBudget": 1,
  "pMinHurdleReve": 0.2,
  "pExpectedReve": [
    [
      [
        0.45,
        0.2
      ],
      [
        0.35,
        0.3
      ]
    ],
    [
      [
        0.55,
        0.1
      ],
      [
        0.2,
        0.45
      ]
    ]
  ],
  "pChannelCosts": [
    0.45,
    0.2
  ],
  "pProductsMinOffersN": [
    1,
    1
  ]
}


### Let's solve the model with the `small_dataset.json` dataset

In [16]:
%%minizinc -m bind --data small_dataset.json
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of var 0..1: vAssignment;

%Constraints
constraint sum(customer in 1..pCustomersN, product in 1..pProductsN, channel in 1..pChannelsN)(vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]) <= pMaxBudget;

constraint forall(product in 1..pProductsN) (
    sum(customer in 1..pCustomersN,
        channel in 1..pChannelsN) (
              vAssignment[customer, product, channel]
    ) >= pProductsMinOffersN[product]);

constraint sum(customer in 1..pCustomersN,
               product in 1..pProductsN,
               channel in 1..pChannelsN)
            (
                vAssignment[customer, product, channel]*pExpectedReve[customer, product, channel]
            ) >= 
            (1+pMinHurdleReve)*sum(customer in 1..pCustomersN,
                                   product in 1..pProductsN,
                                   channel in 1..pChannelsN)
                                    (
                                        vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]
                                    );

solve maximize sum(customer in 1..pCustomersN,
                   product in 1..pProductsN,
                   channel in 1..pChannelsN)
                (
                    vAssignment[customer, product, channel]*
                    pExpectedReve[customer, product, channel]
                );

### Print the solution

In [17]:
for customer in range(len(vAssignment)):
    for product in range(len(vAssignment[customer])):
        for channel in range(len(vAssignment[customer][product])):
            if vAssignment[product][customer][channel] > 0:
                print(f"Offer product:{product+1} to customer {customer+1} using channel {channel+1}")

Offer product:1 to customer 1 using channel 1
Offer product:2 to customer 1 using channel 1
Offer product:1 to customer 2 using channel 1
Offer product:1 to customer 2 using channel 2
Offer product:2 to customer 2 using channel 2


## Augment model (add constraint)

We can see that the model is assigning the same product (id=1) to the same customer (id=2) on both channels. Let's block this by adding extra-constraint.

In [18]:
%%minizinc -m bind --data small_dataset.json
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of var 0..1: vAssignment;

%Constraints
constraint sum(customer in 1..pCustomersN, product in 1..pProductsN, channel in 1..pChannelsN)(vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]) <= pMaxBudget;

constraint forall(product in 1..pProductsN) (
    sum(customer in 1..pCustomersN,
        channel in 1..pChannelsN) (
              vAssignment[customer, product, channel]
    ) >= pProductsMinOffersN[product]);

constraint sum(customer in 1..pCustomersN,
               product in 1..pProductsN,
               channel in 1..pChannelsN)
            (
                vAssignment[customer, product, channel]*pExpectedReve[customer, product, channel]
            ) >= 
            (1+pMinHurdleReve)*sum(customer in 1..pCustomersN,
                                   product in 1..pProductsN,
                                   channel in 1..pChannelsN)
                                    (
                                        vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]
                                    );

%product to customer only via max one channel
constraint forall(customer in 1..pCustomersN,
                  product in 1..pProductsN) (
    sum(channel in 1..pChannelsN)(
        vAssignment[customer, product, channel]
    ) <= 1 
);

solve maximize sum(customer in 1..pCustomersN,
                   product in 1..pProductsN,
                   channel in 1..pChannelsN)
                (
                    vAssignment[customer, product, channel]*
                    pExpectedReve[customer, product, channel]
                );

In [19]:
for customer in range(len(vAssignment)):
    for product in range(len(vAssignment[customer])):
        for channel in range(len(vAssignment[customer][product])):
            if vAssignment[product][customer][channel] > 0:
                print(f"Offer product:{product+1} to customer {customer+1} using channel {channel+1}")

Offer product:1 to customer 1 using channel 1
Offer product:2 to customer 1 using channel 1
Offer product:1 to customer 2 using channel 1
Offer product:2 to customer 2 using channel 2


Now the customer (id=2) gets a product offering via max one channel.

## What if analysis?

### What would be the solution if we reduced budget to 0.5?

In [20]:
%%minizinc -m bind --data small_dataset_whatif.json
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget = 0.5;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of var 0..1: vAssignment;
var float: vTotalMargin;

%Constraints
constraint sum(customer in 1..pCustomersN, product in 1..pProductsN, channel in 1..pChannelsN)(vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]) <= pMaxBudget;

constraint forall(product in 1..pProductsN) (
    sum(customer in 1..pCustomersN,
        channel in 1..pChannelsN) (
              vAssignment[customer, product, channel]
    ) >= pProductsMinOffersN[product]);

constraint sum(customer in 1..pCustomersN,
               product in 1..pProductsN,
               channel in 1..pChannelsN)
            (
                vAssignment[customer, product, channel]*pExpectedReve[customer, product, channel]
            ) >= 
            (1+pMinHurdleReve)*sum(customer in 1..pCustomersN,
                                   product in 1..pProductsN,
                                   channel in 1..pChannelsN)
                                    (
                                        vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]
                                    );

%product to customer only via max one channel
constraint forall(customer in 1..pCustomersN,
                  product in 1..pProductsN) (
    sum(channel in 1..pChannelsN)(
        vAssignment[customer, product, channel]
    ) <= 1 
);

%total margin
constraint vTotalMargin = sum(customer in 1..pCustomersN,
                   product in 1..pProductsN,
                   channel in 1..pChannelsN)
                (
                    vAssignment[customer, product, channel]*
                    pExpectedReve[customer, product, channel]
                );

solve maximize sum(customer in 1..pCustomersN,
                   product in 1..pProductsN,
                   channel in 1..pChannelsN)
                (
                    vAssignment[customer, product, channel]*
                    pExpectedReve[customer, product, channel]
                );

In [21]:
print(f"Total reve {vTotalMargin}")
for customer in range(len(vAssignment)):
    for product in range(len(vAssignment[customer])):
        for channel in range(len(vAssignment[customer][product])):
            if vAssignment[customer][product][channel] > 0:
                print(f"Offer product:{product+1} to customer {customer+1} using channel {channel+1}")

Total reve 1.0
Offer product:1 to customer 2 using channel 1
Offer product:2 to customer 2 using channel 2


### What would be a solution if we set budget to 1.0?

In [22]:
%%minizinc -m bind --data small_dataset_whatif.json
%Parameters
int: pCustomersN;
int: pProductsN;
int: pChannelsN;
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of float: pExpectedReve;
array[1..pChannelsN] of float: pChannelCosts;
float: pMaxBudget = 1.0;
array[1..pProductsN] of int: pProductsMinOffersN;
float: pMinHurdleReve;

%Variables
array[1..pCustomersN, 1..pProductsN, 1..pChannelsN] of var 0..1: vAssignment;
var float: vTotalMargin;

%Constraints
constraint sum(customer in 1..pCustomersN, product in 1..pProductsN, channel in 1..pChannelsN)(vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]) <= pMaxBudget;

constraint forall(product in 1..pProductsN) (
    sum(customer in 1..pCustomersN,
        channel in 1..pChannelsN) (
              vAssignment[customer, product, channel]
    ) >= pProductsMinOffersN[product]);

constraint sum(customer in 1..pCustomersN,
               product in 1..pProductsN,
               channel in 1..pChannelsN)
            (
                vAssignment[customer, product, channel]*pExpectedReve[customer, product, channel]
            ) >= 
            (1+pMinHurdleReve)*sum(customer in 1..pCustomersN,
                                   product in 1..pProductsN,
                                   channel in 1..pChannelsN)
                                    (
                                        vAssignment[customer, product, channel]*pChannelCosts[pChannelsN]
                                    );

%product to customer only via max one channel
constraint forall(customer in 1..pCustomersN,
                  product in 1..pProductsN) (
    sum(channel in 1..pChannelsN)(
        vAssignment[customer, product, channel]
    ) <= 1 
);

%total margin
constraint vTotalMargin = sum(customer in 1..pCustomersN,
                   product in 1..pProductsN,
                   channel in 1..pChannelsN)
                (
                    vAssignment[customer, product, channel]*
                    pExpectedReve[customer, product, channel]
                );

solve maximize sum(customer in 1..pCustomersN,
                   product in 1..pProductsN,
                   channel in 1..pChannelsN)
                (
                    vAssignment[customer, product, channel]*
                    pExpectedReve[customer, product, channel]
                );

In [23]:
print(f"Total reve {vTotalMargin}")
for customer in range(len(vAssignment)):
    for product in range(len(vAssignment[customer])):
        for channel in range(len(vAssignment[customer][product])):
            if vAssignment[customer][product][channel] > 0:
                print(f"Offer product:{product+1} to customer {customer+1} using channel {channel+1}")

Total reve 1.8
Offer product:1 to customer 1 using channel 1
Offer product:2 to customer 1 using channel 1
Offer product:1 to customer 2 using channel 1
Offer product:2 to customer 2 using channel 2


# Technical addon 

## Custom IDE

You can download custom IDE using this [link](https://www.minizinc.org/doc-2.5.5/en/installation.html). It is the easiest way to use *minizinc*.

![mzn_ide](images/mzn-ide-playground4.jpg)

## Using Jupyter Notebook

Extension `iminizinc` allows you to run minizinc models directly in cells.

After loading the extension you can use magic escape `%%minizinc`

In [ ]:
 %load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.5.3
Copyright (C) 2014-2020 Monash University, NICTA, Data61


In [ ]:
n = 10

In [ ]:
%%minizinc
include "alldifferent.mzn";
int: n;
array[1..n] of var 1..n: queens;
constraint alldifferent(queens);
constraint alldifferent([queens[i]+i | i in 1..n]);
constraint alldifferent([queens[i]-i | i in 1..n]);
solve satisfy;

UsageError: Cell magic `%%minizinc` not found.


In [ ]:
%%minizinc?

Docstring:
::

  %minizinc [-v] [-s] [-m {return,bind}] [-a] [-t TIME_LIMIT]
                [--solver SOLVER] [--data [DATA ...]]
                [model ...]

MiniZinc magic

positional arguments:
  model                 Model to solve

options:
  -v, --verbose         Verbose output
  -s, --statistics      Output statistics
  -m <{return,bind}>, --solution-mode <{return,bind}>
                        Whether to return solution(s) or bind them to
                        variables
  -a, --all-solutions   Return all solutions for satisfaction problems,
                        intermediate solutions for optimisation problems.
                        Implies -o.
  -t TIME_LIMIT, --time-limit TIME_LIMIT
                        Time limit in milliseconds (includes compilation and
                        solving)
  --solver SOLVER       Solver to run
  --data <[DATA ...]>   Data files
File:      /workspaces/conf-dss2021/venv/lib/python3.10/site-packages/iminizinc/mzn.py


## Using Python

Package [minizinc](https://minizinc-python.readthedocs.io/en/latest/getting_started.html) gives you possibility to run `minizinc` directly from Python.

In [64]:
## Required only in notebook
import nest_asyncio
nest_asyncio.apply()

In [104]:
from minizinc import Instance, Model, Solver

# Load n-Queens model from file
model = Model("dss-marketing-optimization.mzn")
model.add_file("small_dataset.json", parse_data=True)
# Find the MiniZinc solver configuration for Gecode
gecode = Solver.lookup("gecode")
# Create an Instance of the model for Gecode
instance = Instance(gecode, model)
result = instance.solve()
print(f"Solution objective: {result['objective']}")
for customer in range(instance["pCustomersN"]):
    for product in range(instance["pProductsN"]):
        for channel in range(instance["pChannelsN"]):
            if result["vAssignment"][customer][product][channel] > 0.5:
                print(f"Offer product:{product} to customer {customer} using channel {channel}")

Solution objective: 1.8
Offer product:0 to customer 0 using channel 0
Offer product:1 to customer 0 using channel 0
Offer product:0 to customer 1 using channel 0
Offer product:1 to customer 1 using channel 1
